<a href="https://colab.research.google.com/github/eulerianKnight/semantic_search/blob/main/Semantic_search_example_aurelius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 2.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-xhu8c1dl/farm-haystack_0b0059d23110419383cfcb3130cb9dd3
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-xhu8c1dl/farm-haystack_0b0059d23110419383cfcb3130cb9dd3
  Resolved https://github.com/deepset-ai/haystack.git to commit 21aedc644f701ea57463694c01fb980188ff0e45
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 60.

In [3]:
import json
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever

## Data

In [4]:
!wget https://raw.githubusercontent.com/eulerianKnight/semantic_search/main/meditations.mb.txt

--2022-09-12 11:39:30--  https://raw.githubusercontent.com/eulerianKnight/semantic_search/main/meditations.mb.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243207 (238K) [text/plain]
Saving to: ‘meditations.mb.txt’

meditations.mb.txt  100%[===================>] 237.51K  --.-KB/s    in 0.06s   

2022-09-12 11:39:30 (3.62 MB/s) - ‘meditations.mb.txt’ saved [243207/243207]



In [8]:
with open('meditations.mb.txt', 'r') as f:
    data = f.read()

In [9]:
data = data.split('\n')

In [10]:
data_json = [
    {
    'content': paragraph, 
    'meta': {
        'source': 'meditations.mb', 
    }
} for paragraph in data
]

In [6]:
document_store_faiss = FAISSDocumentStore(faiss_index_factory_str='Flat', 
                                          return_embedding=True)

In [7]:
retriever_faiss = EmbeddingRetriever(document_store_faiss, 
                                     embedding_model='distilroberta-base-msmarco-v2', 
                                     model_format='sentence_transformers')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [11]:
document_store_faiss.write_documents(data_json)

Writing Documents:   0%|          | 0/4300 [00:00<?, ?it/s]

In [12]:
document_store_faiss.update_embeddings(retriever=retriever_faiss)

Updating Embedding:   0%|          | 0/3760 [00:00<?, ? docs/s]

Batches:   0%|          | 0/118 [00:00<?, ?it/s]

In [13]:
def get_results(query, retriever,  n_result=1):
  return [(item.content, item.to_dict()['meta']) for item in retriever.retrieve(query, top_k=n_result)]

In [15]:
query = 'fortne is separated'
print('Result: ')
res = get_results(query, retriever_faiss)
for r in res:
  print(r)

Result: 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('fortune is not separated from nature or without an interweaving and', {'source': 'meditations.mb', 'vector_id': '261'})
